In [1]:
from pyspark.sql import SparkSession

from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession \
    .builder \
    .appName("pyspark_Obl_NBA_rfn") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2025-07-08T13:29:32,564 WARN [Thread-4] org.apache.hadoop.util.NativeCodeLoader - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## CARGA DE LOS CSV

In [2]:
df_game = spark.read.csv('/data_obligatorio_NBA/lnd/game.csv', header=True, sep=',')
df_game_summary = spark.read.csv('/data_obligatorio_NBA/lnd/game_summary.csv', header=True, sep=',')
df_common_player_info = spark.read.csv('/data_obligatorio_NBA/lnd/common_player_info.csv', header=True, sep=',')
df_play_by_play_pts = spark.read.csv('/data_obligatorio_NBA/lnd/play_by_play_pts.csv', header=True, sep=',')
df_player = spark.read.csv('/data_obligatorio_NBA/lnd/player.csv', header=True, sep=',')
df_team = spark.read.csv('/data_obligatorio_NBA/lnd/team.csv', header=True, sep=',')

## IMPRESION DE LOS ATRIBUTOS

In [3]:
df_game.printSchema()

root
 |-- team_id_home: string (nullable = true)
 |-- game_id: string (nullable = true)
 |-- wl_home: string (nullable = true)
 |-- pts_home: string (nullable = true)
 |-- team_id_away: string (nullable = true)
 |-- wl_away: string (nullable = true)
 |-- pts_away: string (nullable = true)



In [4]:
df_game_summary.printSchema()

root
 |-- game_id: string (nullable = true)
 |-- home_team_id: string (nullable = true)
 |-- visitor_team_id: string (nullable = true)
 |-- season: string (nullable = true)



In [5]:
df_common_player_info.printSchema()

root
 |-- person_id: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- birthdate: string (nullable = true)
 |-- school: string (nullable = true)
 |-- country: string (nullable = true)
 |-- jersey: string (nullable = true)
 |-- position: string (nullable = true)
 |-- rosterstatus: string (nullable = true)
 |-- games_played_current_season_flag: string (nullable = true)
 |-- team_id: string (nullable = true)
 |-- team_name: string (nullable = true)
 |-- team_city: string (nullable = true)
 |-- from_year: string (nullable = true)
 |-- to_year: string (nullable = true)



In [6]:
df_play_by_play_pts.printSchema()

root
 |-- game_id: string (nullable = true)
 |-- eventnum: string (nullable = true)
 |-- period: string (nullable = true)
 |-- wctimestring: string (nullable = true)
 |-- homedescription: string (nullable = true)
 |-- visitordescription: string (nullable = true)
 |-- score: string (nullable = true)
 |-- scoremargin: string (nullable = true)
 |-- player1_id: string (nullable = true)
 |-- player1_name: string (nullable = true)
 |-- player1_team_id: string (nullable = true)



In [7]:
df_player.printSchema()

root
 |-- id: string (nullable = true)
 |-- full_name: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- is_active: string (nullable = true)



In [8]:
df_team.printSchema()

root
 |-- id: string (nullable = true)
 |-- full_name: string (nullable = true)
 |-- state: string (nullable = true)
 |-- year_founded: string (nullable = true)



## CREACION DE SCHEMAS, CONVIRTIENDO LOS ATRIBUTOS A TIPOS CORRESPONDIENTES

In [9]:
schema_game = StructType([
    StructField("team_id_home", IntegerType(), True),
    StructField("game_id", IntegerType(), True),
    StructField("wl_home", StringType(), True),
    StructField("pts_home", DoubleType(), True),
    StructField("team_id_away", IntegerType(), True),
    StructField("wl_away", StringType(), True),
    StructField("pts_away", DoubleType(), True)
])

schema_game_summary = StructType([
    StructField("game_id", IntegerType(), True),
    StructField("home_team_id", IntegerType(), True),
    StructField("visitor_team_id", IntegerType(), True),
    StructField("season", IntegerType(), True)
])

schema_common_player_info = StructType([
    StructField("person_id", IntegerType(), True),
    StructField("first_name", StringType(), True),
    StructField("last_name", StringType(), True),
    StructField("birthdate", TimestampType(), True),
    StructField("school", StringType(), True),
    StructField("country", StringType(), True),
    StructField("jersey", StringType(), True),
    StructField("position", StringType(), True),
    StructField("rosterstatus", StringType(), True),
    StructField("games_played_current_season_flag", StringType(), True),
    StructField("team_id", IntegerType(), True),
    StructField("team_name", StringType(), True),
    StructField("team_city", StringType(), True),
    StructField("from_year", StringType(), True),
    StructField("to_year", StringType(), True)
])

schema_play_by_play_pts = StructType([
    StructField("game_id", IntegerType(), True),
    StructField("eventnum", IntegerType(), True),
    StructField("period", IntegerType(), True),
    StructField("wctimestring", StringType(), True),
    StructField("homedescription", StringType(), True),
    StructField("visitordescription", StringType(), True),
    StructField("score", StringType(), True),
    StructField("scoremargin", StringType(), True),
    StructField("player1_id", IntegerType(), True),
    StructField("player1_name", StringType(), True),
    StructField("player1_team_id", StringType(), True),
])

schema_player = StructType([
    StructField("id", IntegerType(), True),
    StructField("full_name", StringType(), True),
    StructField("first_name", StringType(), True),
    StructField("last_name", StringType(), True),
    StructField("is_active", IntegerType(), True)
])

schema_team = StructType([
    StructField("id", IntegerType(), True),
    StructField("full_name", StringType(), True),
    StructField("state", StringType(), True),
    StructField("year_founded", StringType(), True)
])


# Lectura del archivo CSV con schema
df_game = spark.read.csv('/data_obligatorio_NBA/lnd/game.csv', header=True, sep=',', schema=schema_game)
df_game_summary = spark.read.csv('/data_obligatorio_NBA/lnd/game_summary.csv', header=True, sep=',', schema=schema_game_summary)
df_common_player_info = spark.read.csv('/data_obligatorio_NBA/lnd/common_player_info.csv', header=True, sep=',', schema=schema_common_player_info)
df_play_by_play_pts = spark.read.csv('/data_obligatorio_NBA/lnd/play_by_play_pts.csv', header=True, sep=',', schema=schema_play_by_play_pts)
df_player = spark.read.csv('/data_obligatorio_NBA/lnd/player.csv', header=True, sep=',', schema=schema_player)
df_team = spark.read.csv('/data_obligatorio_NBA/lnd/team.csv', header=True, sep=',', schema=schema_team)

## CONVERSION DE TIPOS DE DATOS MAL REPRESENTADOS EN LA FUENTE DE DATOS

In [10]:
df_game.select("pts_home", "pts_away").show(10)
df_game = df_game.withColumn("pts_home", df_game["pts_home"].cast("int"))
df_game = df_game.withColumn("pts_away", df_game["pts_away"].cast("int"))
df_game.select("pts_home", "pts_away").show(10)

+--------+--------+
|pts_home|pts_away|
+--------+--------+
|    66.0|    68.0|
|    56.0|    51.0|
|    59.0|    53.0|
|    63.0|    47.0|
|    33.0|    50.0|
|    71.0|    60.0|
|    56.0|    71.0|
|    55.0|    57.0|
|    49.0|    53.0|
|    73.0|    65.0|
+--------+--------+
only showing top 10 rows

+--------+--------+
|pts_home|pts_away|
+--------+--------+
|      66|      68|
|      56|      51|
|      59|      53|
|      63|      47|
|      33|      50|
|      71|      60|
|      56|      71|
|      55|      57|
|      49|      53|
|      73|      65|
+--------+--------+
only showing top 10 rows



In [11]:
df_common_player_info.select("from_year", "to_year").show(10)
df_common_player_info = df_common_player_info.withColumn("from_year", df_common_player_info["from_year"].cast("int"))
df_common_player_info = df_common_player_info.withColumn("to_year", df_common_player_info["to_year"].cast("int"))
df_common_player_info.select("from_year", "to_year").show(10)

+---------+-------+
|from_year|to_year|
+---------+-------+
|   1990.0| 1994.0|
|   1968.0| 1977.0|
|   1969.0| 1988.0|
|   1996.0| 2007.0|
|   1956.0| 1956.0|
|   1946.0| 1947.0|
|   2016.0| 2018.0|
|   2020.0| 2023.0|
|   2012.0| 2018.0|
|   1975.0| 1987.0|
+---------+-------+
only showing top 10 rows

+---------+-------+
|from_year|to_year|
+---------+-------+
|     1990|   1994|
|     1968|   1977|
|     1969|   1988|
|     1996|   2007|
|     1956|   1956|
|     1946|   1947|
|     2016|   2018|
|     2020|   2023|
|     2012|   2018|
|     1975|   1987|
+---------+-------+
only showing top 10 rows



In [12]:
df_play_by_play_pts.select("player1_team_id").show(10)
df_play_by_play_pts = df_play_by_play_pts.withColumn("player1_team_id", df_play_by_play_pts["player1_team_id"].cast("int"))
df_play_by_play_pts.select("player1_team_id").show(10)

+---------------+
|player1_team_id|
+---------------+
|   1610612747.0|
|   1610612747.0|
|   1610612756.0|
|   1610612756.0|
|   1610612756.0|
|   1610612747.0|
|   1610612747.0|
|   1610612747.0|
|   1610612756.0|
|   1610612756.0|
+---------------+
only showing top 10 rows

+---------------+
|player1_team_id|
+---------------+
|     1610612747|
|     1610612747|
|     1610612756|
|     1610612756|
|     1610612756|
|     1610612747|
|     1610612747|
|     1610612747|
|     1610612756|
|     1610612756|
+---------------+
only showing top 10 rows



In [13]:
df_team.select("year_founded").show(10)
df_team = df_team.withColumn("year_founded", df_team["year_founded"].cast("int"))
df_team.select("year_founded").show(10)

+------------+
|year_founded|
+------------+
|      1949.0|
|      1946.0|
|      1970.0|
|      2002.0|
|      1966.0|
|      1980.0|
|      1976.0|
|      1946.0|
|      1967.0|
|      1970.0|
+------------+
only showing top 10 rows

+------------+
|year_founded|
+------------+
|        1949|
|        1946|
|        1970|
|        2002|
|        1966|
|        1980|
|        1976|
|        1946|
|        1967|
|        1970|
+------------+
only showing top 10 rows



## GESTION DE DUPLICADOS

###  ELIMINACION DE DUPLICADOS

In [14]:
# eliminar duplicados
df_game = df_game.dropDuplicates()
df_game_summary = df_game_summary.dropDuplicates()
df_common_player_info = df_common_player_info.dropDuplicates()
df_play_by_play_pts = df_play_by_play_pts.dropDuplicates()
df_player = df_player.dropDuplicates()
df_team = df_team.dropDuplicates()

### VERIFICACION DE LA ELIMINACION DE LOS DUPLICADOS (NO DEBE HABER RETORNO)

In [15]:
df_game.groupBy("game_id") \
    .count() \
    .filter("count > 1") \
    .orderBy("count", ascending=False) \
    .show()

[Stage 14:>                                                         (0 + 1) / 1]

+-------+-----+
|game_id|count|
+-------+-----+
+-------+-----+



In [16]:
df_game_summary.groupBy("game_id") \
    .count() \
    .filter("count > 1") \
    .orderBy("count", ascending=False) \
    .show()

+-------+-----+
|game_id|count|
+-------+-----+
+-------+-----+



In [17]:
df_common_player_info.groupBy("person_id") \
    .count() \
    .filter("count > 1") \
    .orderBy("count", ascending=False) \
    .show()

+---------+-----+
|person_id|count|
+---------+-----+
+---------+-----+



In [18]:
df_play_by_play_pts.groupBy("game_id", "eventnum") \
    .count() \
    .filter("count > 1") \
    .orderBy("count", ascending=False) \
    .show()

[Stage 34:>                                                         (0 + 2) / 5]

2025-07-08T13:30:20,541 WARN [Executor task launch worker for task 0.0 in stage 34.0 (TID 26)] org.apache.spark.sql.catalyst.expressions.RowBasedKeyValueBatch - Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
2025-07-08T13:30:20,629 WARN [Executor task launch worker for task 0.0 in stage 34.0 (TID 26)] org.apache.spark.sql.catalyst.expressions.RowBasedKeyValueBatch - Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
2025-07-08T13:30:20,669 WARN [Executor task launch worker for task 1.0 in stage 34.0 (TID 27)] org.apache.spark.sql.catalyst.expressions.RowBasedKeyValueBatch - Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
2025-07-08T13:30:20,726 WARN [Executor task launch worker for task 1.0 in stage 34.0 (TID 27)] org.apache.spark.sql.catalyst.expressions.RowBasedKeyValueBatch - Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


[Stage 34:=======================>                                  (2 + 2) / 5]

2025-07-08T13:30:25,843 WARN [Executor task launch worker for task 3.0 in stage 34.0 (TID 29)] org.apache.spark.sql.catalyst.expressions.RowBasedKeyValueBatch - Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
2025-07-08T13:30:25,865 WARN [Executor task launch worker for task 2.0 in stage 34.0 (TID 28)] org.apache.spark.sql.catalyst.expressions.RowBasedKeyValueBatch - Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
2025-07-08T13:30:25,905 WARN [Executor task launch worker for task 3.0 in stage 34.0 (TID 29)] org.apache.spark.sql.catalyst.expressions.RowBasedKeyValueBatch - Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


[Stage 37:>                                                         (0 + 2) / 2]

+-------+--------+-----+
|game_id|eventnum|count|
+-------+--------+-----+
+-------+--------+-----+



In [19]:
df_player.groupBy("id") \
    .count() \
    .filter("count > 1") \
    .orderBy("count", ascending=False) \
    .show()

+---+-----+
| id|count|
+---+-----+
+---+-----+



In [20]:
df_team.groupBy("id") \
    .count() \
    .filter("count > 1") \
    .orderBy("count", ascending=False) \
    .show()

+---+-----+
| id|count|
+---+-----+
+---+-----+



## GESTION DE NULOS

### ELIMINACION DE DATOS NULOS

In [21]:
for field in df_game.schema.fields:
    name = field.name
    dtype = field.dataType
    
    if isinstance(dtype, StringType):
        default_value = lit("Undefined")
        condition = col(name).isNull() | (trim(col(name)) == "")
    elif isinstance(dtype, IntegerType):
        default_value = lit(-1)
        condition = col(name).isNull()

    df_game = df_game.withColumn(
        name,
        when(col(name).isNull() |
             ((col(name).cast("string").isNotNull()) & (trim(col(name)) == "")), default_value)
        .otherwise(col(name))
    )

In [22]:
for field in df_common_player_info.schema.fields:
    name = field.name
    dtype = field.dataType
    
    if isinstance(dtype, StringType):
        default_value = lit("Undefined")
        condition = col(name).isNull() | (trim(col(name)) == "")
    elif isinstance(dtype, IntegerType):
        default_value = lit(-1)
        condition = col(name).isNull()

    df_common_player_info = df_common_player_info.withColumn(
        name,
        when(col(name).isNull() |
             ((col(name).cast("string").isNotNull()) & (trim(col(name)) == "")), default_value)
        .otherwise(col(name))
    )

In [23]:
for field in df_play_by_play_pts.schema.fields:
    name = field.name
    dtype = field.dataType
    
    if isinstance(dtype, StringType):
        default_value = lit("Undefined")
        condition = col(name).isNull() | (trim(col(name)) == "")
    elif isinstance(dtype, IntegerType):
        default_value = lit(-1)
        condition = col(name).isNull()

    df_play_by_play_pts = df_play_by_play_pts.withColumn(
        name,
        when(col(name).isNull() |
             ((col(name).cast("string").isNotNull()) & (trim(col(name)) == "")), default_value)
        .otherwise(col(name))
    )

### VERIFICACION DE LA ELIMINACION DE NULOS

In [24]:
null_or_empty_counts = df_game.select([
    sum(
        when(
            col(c).isNull() | 
            ((col(c).cast("string").isNotNull()) & (trim(col(c)) == "")), 1
        ).otherwise(0)
    ).alias(c)
    for c in df_game.columns
])

null_or_empty_counts.show()


+------------+-------+-------+--------+------------+-------+--------+
|team_id_home|game_id|wl_home|pts_home|team_id_away|wl_away|pts_away|
+------------+-------+-------+--------+------------+-------+--------+
|           0|      0|      0|       0|           0|      0|       0|
+------------+-------+-------+--------+------------+-------+--------+



In [25]:
null_or_empty_counts = df_common_player_info.select([
    sum(
        when(
            col(c).isNull() | 
            ((col(c).cast("string").isNotNull()) & (trim(col(c)) == "")), 1
        ).otherwise(0)
    ).alias(c)
    for c in df_common_player_info.columns
])

null_or_empty_counts.show()


+---------+----------+---------+---------+------+-------+------+--------+------------+--------------------------------+-------+---------+---------+---------+-------+
|person_id|first_name|last_name|birthdate|school|country|jersey|position|rosterstatus|games_played_current_season_flag|team_id|team_name|team_city|from_year|to_year|
+---------+----------+---------+---------+------+-------+------+--------+------------+--------------------------------+-------+---------+---------+---------+-------+
|        0|         0|        0|        0|     0|      0|     0|       0|           0|                               0|      0|        0|        0|        0|      0|
+---------+----------+---------+---------+------+-------+------+--------+------------+--------------------------------+-------+---------+---------+---------+-------+



In [26]:
null_or_empty_counts = df_play_by_play_pts.select([
    sum(
        when(
            col(c).isNull() | 
            ((col(c).cast("string").isNotNull()) & (trim(col(c)) == "")), 1
        ).otherwise(0)
    ).alias(c)
    for c in df_play_by_play_pts.columns
])

null_or_empty_counts.show()


[Stage 64:==============================================>           (4 + 1) / 5]

+-------+--------+------+------------+---------------+------------------+-----+-----------+----------+------------+---------------+
|game_id|eventnum|period|wctimestring|homedescription|visitordescription|score|scoremargin|player1_id|player1_name|player1_team_id|
+-------+--------+------+------------+---------------+------------------+-----+-----------+----------+------------+---------------+
|      0|       0|     0|           0|              0|                 0|    0|          0|         0|           0|              0|
+-------+--------+------+------------+---------------+------------------+-----+-----------+----------+------------+---------------+



## VALIDACION DE UNICIDAD DE CLAVES

In [27]:
def unique_key(df, col):
    tot_rows = df.count()
    primary_row = df.select(col).distinct().count()
    
    if tot_rows == primary_row:
        print(f"UNIQUE KEY")
    else:
        print(f"NOT UNIQUE KEY")

In [28]:
unique_key(df_game, "game_id")

UNIQUE KEY


In [29]:
unique_key(df_game_summary, "game_id")

UNIQUE KEY


In [30]:
unique_key(df_common_player_info, "person_id")

UNIQUE KEY


In [31]:
unique_key(df_play_by_play_pts, ["game_id", "eventnum"])

[Stage 112:>                                                        (0 + 2) / 2]

UNIQUE KEY


In [32]:
unique_key(df_player, "id")

UNIQUE KEY


In [33]:
unique_key(df_team, "id")

UNIQUE KEY


## IMPRESION DE LAS COLUMNAS DE LOS DF FINALES CON SUS TIPOS

In [34]:
df_game.printSchema()

root
 |-- team_id_home: integer (nullable = true)
 |-- game_id: integer (nullable = true)
 |-- wl_home: string (nullable = true)
 |-- pts_home: integer (nullable = true)
 |-- team_id_away: integer (nullable = true)
 |-- wl_away: string (nullable = true)
 |-- pts_away: integer (nullable = true)



In [35]:
df_game_summary.printSchema()

root
 |-- game_id: integer (nullable = true)
 |-- home_team_id: integer (nullable = true)
 |-- visitor_team_id: integer (nullable = true)
 |-- season: integer (nullable = true)



In [36]:
df_common_player_info.printSchema()

root
 |-- person_id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- birthdate: string (nullable = true)
 |-- school: string (nullable = true)
 |-- country: string (nullable = true)
 |-- jersey: string (nullable = true)
 |-- position: string (nullable = true)
 |-- rosterstatus: string (nullable = true)
 |-- games_played_current_season_flag: string (nullable = true)
 |-- team_id: integer (nullable = true)
 |-- team_name: string (nullable = true)
 |-- team_city: string (nullable = true)
 |-- from_year: integer (nullable = true)
 |-- to_year: integer (nullable = true)



In [37]:
df_play_by_play_pts.printSchema()

root
 |-- game_id: integer (nullable = true)
 |-- eventnum: integer (nullable = true)
 |-- period: integer (nullable = true)
 |-- wctimestring: string (nullable = true)
 |-- homedescription: string (nullable = true)
 |-- visitordescription: string (nullable = true)
 |-- score: string (nullable = true)
 |-- scoremargin: string (nullable = true)
 |-- player1_id: integer (nullable = true)
 |-- player1_name: string (nullable = true)
 |-- player1_team_id: integer (nullable = true)



In [38]:
df_player.printSchema()

root
 |-- id: integer (nullable = true)
 |-- full_name: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- is_active: integer (nullable = true)



In [39]:
df_team.printSchema()

root
 |-- id: integer (nullable = true)
 |-- full_name: string (nullable = true)
 |-- state: string (nullable = true)
 |-- year_founded: integer (nullable = true)



## VISUALIZACION DE LOS PRIMEROS DATOS DE LOS DF FINALES

In [40]:
df_game.show(10)

+------------+--------+-------+--------+------------+-------+--------+
|team_id_home| game_id|wl_home|pts_home|team_id_away|wl_away|pts_away|
+------------+--------+-------+--------+------------+-------+--------+
|  1610610026|24600156|      L|      74|  1610610032|      W|      78|
|  1610612738|24600187|      L|      57|  1610610036|      W|      80|
|  1610612747|24800213|      W|      89|  1610612744|      L|      71|
|  1610612765|24800254|      W|      89|  1610612744|      L|      87|
|  1610610029|24800326|      L|      80|  1610610034|      W|      81|
|  1610610025|44800212|      L|      85|  1610612747|      W|     101|
|  1610610036|25000004|      W|     100|  1610610030|      L|      84|
|  1610612755|25000213|      L|      86|  1610612744|      W|      96|
|  1610612738|25000260|      W|      85|  1610612737|      L|      70|
|  1610610024|25000270|      L|      72|  1610612755|      W|      80|
+------------+--------+-------+--------+------------+-------+--------+
only s

In [41]:
df_game_summary.show(10)

+--------+------------+---------------+------+
| game_id|home_team_id|visitor_team_id|season|
+--------+------------+---------------+------+
|24600033|  1610610031|     1610610035|  1946|
|24600302|  1610612738|     1610610026|  1946|
|44600212|  1610612752|     1610612744|  1946|
|24800250|  1610612747|     1610612758|  1948|
|24800253|  1610610029|     1610612744|  1948|
|24900180|  1610610024|     1610612747|  1949|
|25000079|  1610612737|     1610610024|  1950|
|25100011|  1610612758|     1610612747|  1951|
|25100142|  1610612752|     1610612738|  1951|
|25100151|  1610612744|     1610610024|  1951|
+--------+------------+---------------+------+
only showing top 10 rows



In [42]:
df_common_player_info.select("person_id", "first_name", "last_name", "birthdate", "school", "country", "jersey", "position").show(10)

+---------+----------+---------+-------------------+------------------+-------+------+--------------+
|person_id|first_name|last_name|          birthdate|            school|country|jersey|      position|
+---------+----------+---------+-------------------+------------------+-------+------+--------------+
|   201587|   Nicolas|    Batum|1988-12-14 00:00:00|           Le Mans| France|    33| Guard-Forward|
|  1628418|    Thomas|   Bryant|1997-07-31 00:00:00|           Indiana|    USA|    13|Center-Forward|
|    76590|      Leon|  Douglas|1954-08-26 00:00:00|           Alabama|    USA|    13|        Center|
|  1629164|   Brandon|  Goodwin|1995-10-02 00:00:00|Florida Gulf Coast|    USA|    00|         Guard|
|     2580|    Travis|   Hansen|1978-04-15 00:00:00|     Brigham Young|    USA|     6| Guard-Forward|
|  1626209|     Mario|  Hezonja|1995-02-25 00:00:00|      FC Barcelona|Croatia|    44| Forward-Guard|
|    77194|     Robin|    Jones|1954-02-02 00:00:00|         St. Louis|    USA|   

In [43]:
df_common_player_info.select("rosterstatus", "games_played_current_season_flag", "team_id", "team_name", "team_city", "from_year", "to_year").show(10)

+------------+--------------------------------+----------+-------------+-----------+---------+-------+
|rosterstatus|games_played_current_season_flag|   team_id|    team_name|  team_city|from_year|to_year|
+------------+--------------------------------+----------+-------------+-----------+---------+-------+
|      Active|                               N|1610612746|     Clippers|         LA|     2008|   2023|
|      Active|                               N|1610612748|         Heat|      Miami|     2017|   2023|
|    Inactive|                               N|1610612765|      Pistons|    Detroit|     1976|   1982|
|    Inactive|                               N|1610612739|    Cavaliers|  Cleveland|     2018|   2021|
|    Inactive|                               N|1610612737|        Hawks|    Atlanta|     2003|   2003|
|    Inactive|                               N|1610612753|        Magic|    Orlando|     2015|   2019|
|    Inactive|                               N|1610612757|Trail Blazers| 

In [44]:
df_play_by_play_pts.show(10)

[Stage 154:>                                                        (0 + 1) / 1]

+--------+--------+------+------------+--------------------+--------------------+---------+-----------+----------+--------------------+---------------+
| game_id|eventnum|period|wctimestring|     homedescription|  visitordescription|    score|scoremargin|player1_id|        player1_name|player1_team_id|
+--------+--------+------+------------+--------------------+--------------------+---------+-----------+----------+--------------------+---------------+
|20200222|       8|     1|   Undefined|World Peace Free ...|           Undefined|    0 - 1|          1|      1897|   Metta World Peace|     1610612754|
|20000814|     156|     2|  -10:0-14 A|Manning  Layup (4...|           Undefined|  31 - 31|        TIE|       330|       Danny Manning|     1610612762|
|20400395|     471|     4|    10:00 PM|Laettner Free Thr...|           Undefined|100 - 107|          7|       363|  Christian Laettner|     1610612748|
|20200134|     420|     4|    10:00 PM|Garnett  Layup (2...|           Undefined|  85 - 

In [45]:
df_player.show(10)

+-------+----------------+----------+---------+---------+
|     id|       full_name|first_name|last_name|is_active|
+-------+----------------+----------+---------+---------+
|1630175|    Cole Anthony|      Cole|  Anthony|        1|
|  76054|  Nate Archibald|      Nate|Archibald|        0|
|1629633|  Jarrett Culver|   Jarrett|   Culver|        1|
|   2223|Samuel Dalembert|    Samuel|Dalembert|        0|
| 201942|   DeMar DeRozan|     DeMar|  DeRozan|        1|
|  76599|      Larry Drew|     Larry|     Drew|        0|
|1627770|      Kay Felder|       Kay|   Felder|        0|
| 101155|      Ryan Gomes|      Ryan|    Gomes|        0|
|  77112|  Winfred Jacobs|   Winfred|   Jacobs|        0|
|  77129|      Les Jepsen|       Les|   Jepsen|        0|
+-------+----------------+----------+---------+---------+
only showing top 10 rows



In [46]:
df_team.show(10)

+----------+--------------------+--------------+------------+
|        id|           full_name|         state|year_founded|
+----------+--------------------+--------------+------------+
|1610612741|       Chicago Bulls|      Illinois|        1966|
|1610612752|     New York Knicks|      New York|        1946|
|1610612751|       Brooklyn Nets|      New York|        1976|
|1610612738|      Boston Celtics| Massachusetts|        1946|
|1610612762|           Utah Jazz|          Utah|        1974|
|1610612750|Minnesota Timberw...|     Minnesota|        1989|
|1610612766|   Charlotte Hornets|North Carolina|        1988|
|1610612748|          Miami Heat|       Florida|        1988|
|1610612758|    Sacramento Kings|    California|        1948|
|1610612757|Portland Trail Bl...|        Oregon|        1970|
+----------+--------------------+--------------+------------+
only showing top 10 rows



## GUARDADO DE LOS DF EN FORMATO PARQUET

In [47]:
df_game.write \
  .mode("overwrite") \
  .parquet("/data_obligatorio_NBA/rfn/game_pqt")

df_game_summary.write \
  .mode("overwrite") \
  .parquet("/data_obligatorio_NBA/rfn/game_sum_pqt")

df_common_player_info.write \
  .mode("overwrite") \
  .parquet("/data_obligatorio_NBA/rfn/com_play_info_pqt")

df_play_by_play_pts.write \
  .mode("overwrite") \
  .parquet("/data_obligatorio_NBA/rfn/play_pts_pqt")

df_player.write \
  .mode("overwrite") \
  .parquet("/data_obligatorio_NBA/rfn/player_pqt")

df_team.write \
  .mode("overwrite") \
  .parquet("/data_obligatorio_NBA/rfn/team_pqt")

## LECTURA DE LOS DF EN FORMATO PARQUET PARA COMPROBAR SU GUARDADO

In [48]:
df_parquet = spark.read.parquet("/data_obligatorio_NBA/rfn/game_pqt")
df_parquet.show(10)

+------------+--------+-------+--------+------------+-------+--------+
|team_id_home| game_id|wl_home|pts_home|team_id_away|wl_away|pts_away|
+------------+--------+-------+--------+------------+-------+--------+
|  1610610026|24600156|      L|      74|  1610610032|      W|      78|
|  1610612738|24600187|      L|      57|  1610610036|      W|      80|
|  1610612747|24800213|      W|      89|  1610612744|      L|      71|
|  1610612765|24800254|      W|      89|  1610612744|      L|      87|
|  1610610029|24800326|      L|      80|  1610610034|      W|      81|
|  1610610025|44800212|      L|      85|  1610612747|      W|     101|
|  1610610036|25000004|      W|     100|  1610610030|      L|      84|
|  1610612755|25000213|      L|      86|  1610612744|      W|      96|
|  1610612738|25000260|      W|      85|  1610612737|      L|      70|
|  1610610024|25000270|      L|      72|  1610612755|      W|      80|
+------------+--------+-------+--------+------------+-------+--------+
only s

In [49]:
df_parquet = spark.read.parquet("/data_obligatorio_NBA/rfn/game_sum_pqt")
df_parquet.show(10)

+--------+------------+---------------+------+
| game_id|home_team_id|visitor_team_id|season|
+--------+------------+---------------+------+
|24600033|  1610610031|     1610610035|  1946|
|24600302|  1610612738|     1610610026|  1946|
|44600212|  1610612752|     1610612744|  1946|
|24800250|  1610612747|     1610612758|  1948|
|24800253|  1610610029|     1610612744|  1948|
|24900180|  1610610024|     1610612747|  1949|
|25000079|  1610612737|     1610610024|  1950|
|25100011|  1610612758|     1610612747|  1951|
|25100142|  1610612752|     1610612738|  1951|
|25100151|  1610612744|     1610610024|  1951|
+--------+------------+---------------+------+
only showing top 10 rows



In [50]:
df_parquet = spark.read.parquet("/data_obligatorio_NBA/rfn/com_play_info_pqt")
df_parquet.show(10)

+---------+----------+---------+-------------------+------------------+-------+------+--------------+------------+--------------------------------+----------+-------------+-----------+---------+-------+
|person_id|first_name|last_name|          birthdate|            school|country|jersey|      position|rosterstatus|games_played_current_season_flag|   team_id|    team_name|  team_city|from_year|to_year|
+---------+----------+---------+-------------------+------------------+-------+------+--------------+------------+--------------------------------+----------+-------------+-----------+---------+-------+
|   201587|   Nicolas|    Batum|1988-12-14 00:00:00|           Le Mans| France|    33| Guard-Forward|      Active|                               N|1610612746|     Clippers|         LA|     2008|   2023|
|  1628418|    Thomas|   Bryant|1997-07-31 00:00:00|           Indiana|    USA|    13|Center-Forward|      Active|                               N|1610612748|         Heat|      Miami|    

In [51]:
df_parquet = spark.read.parquet("/data_obligatorio_NBA/rfn/play_pts_pqt")
df_parquet.show(10)

+--------+--------+------+------------+--------------------+--------------------+---------+-----------+----------+--------------------+---------------+
| game_id|eventnum|period|wctimestring|     homedescription|  visitordescription|    score|scoremargin|player1_id|        player1_name|player1_team_id|
+--------+--------+------+------------+--------------------+--------------------+---------+-----------+----------+--------------------+---------------+
|20200222|       8|     1|   Undefined|World Peace Free ...|           Undefined|    0 - 1|          1|      1897|   Metta World Peace|     1610612754|
|20000814|     156|     2|  -10:0-14 A|Manning  Layup (4...|           Undefined|  31 - 31|        TIE|       330|       Danny Manning|     1610612762|
|20400395|     471|     4|    10:00 PM|Laettner Free Thr...|           Undefined|100 - 107|          7|       363|  Christian Laettner|     1610612748|
|20200134|     420|     4|    10:00 PM|Garnett  Layup (2...|           Undefined|  85 - 

In [52]:
df_parquet = spark.read.parquet("/data_obligatorio_NBA/rfn/player_pqt")
df_parquet.show(10)

+-------+----------------+----------+---------+---------+
|     id|       full_name|first_name|last_name|is_active|
+-------+----------------+----------+---------+---------+
|1630175|    Cole Anthony|      Cole|  Anthony|        1|
|  76054|  Nate Archibald|      Nate|Archibald|        0|
|1629633|  Jarrett Culver|   Jarrett|   Culver|        1|
|   2223|Samuel Dalembert|    Samuel|Dalembert|        0|
| 201942|   DeMar DeRozan|     DeMar|  DeRozan|        1|
|  76599|      Larry Drew|     Larry|     Drew|        0|
|1627770|      Kay Felder|       Kay|   Felder|        0|
| 101155|      Ryan Gomes|      Ryan|    Gomes|        0|
|  77112|  Winfred Jacobs|   Winfred|   Jacobs|        0|
|  77129|      Les Jepsen|       Les|   Jepsen|        0|
+-------+----------------+----------+---------+---------+
only showing top 10 rows



In [53]:
df_parquet = spark.read.parquet("/data_obligatorio_NBA/rfn/team_pqt")
df_parquet.show(10)

+----------+--------------------+--------------+------------+
|        id|           full_name|         state|year_founded|
+----------+--------------------+--------------+------------+
|1610612741|       Chicago Bulls|      Illinois|        1966|
|1610612752|     New York Knicks|      New York|        1946|
|1610612751|       Brooklyn Nets|      New York|        1976|
|1610612738|      Boston Celtics| Massachusetts|        1946|
|1610612762|           Utah Jazz|          Utah|        1974|
|1610612750|Minnesota Timberw...|     Minnesota|        1989|
|1610612766|   Charlotte Hornets|North Carolina|        1988|
|1610612748|          Miami Heat|       Florida|        1988|
|1610612758|    Sacramento Kings|    California|        1948|
|1610612757|Portland Trail Bl...|        Oregon|        1970|
+----------+--------------------+--------------+------------+
only showing top 10 rows

